# Project 1
---

### Analysis

- Here is where the analysis will be.

## Setup
--------

In [20]:
# Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as st

In [21]:
#Import master data file
master_data_df = pd.read_csv("data.csv")
master_data_df.head(1)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,Unnamed: 60
0,Arab World,ARB,% of females ages 15-49 having comprehensive c...,SH.HIV.KNOW.FE.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
#Limit years for cleaned data to 2006-2015
cleaned_data_df = master_data_df [["Country Name", "Indicator Name", "2006", "2007", "2008", "2009", "2010", "2011", "2012", "2013", "2014", "2015"]]

#Limit countries for cleaned data to 10 largest in the world
cleaned_data_df = cleaned_data_df .loc[(cleaned_data_df ["Country Name"] == "China") |
                     (cleaned_data_df ["Country Name"] == "India") |
                     (cleaned_data_df ["Country Name"] == "United States") |
                     (cleaned_data_df ["Country Name"] == "Indonesia") |
                     (cleaned_data_df ["Country Name"] == "Brazil") |
                     (cleaned_data_df ["Country Name"] == "Pakistan") |
                     (cleaned_data_df ["Country Name"] == "Nigeria") |
                     (cleaned_data_df ["Country Name"] == "Bangladesh") |
                     (cleaned_data_df ["Country Name"] == "Russia") |
                     (cleaned_data_df ["Country Name"] == "Mexico")
                     , :]

#Limit indicators to those relevant for planned analysis
cleaned_data_df = cleaned_data_df .loc[(cleaned_data_df ["Indicator Name"] == "Mortality rate, adult, male (per 1,000 male adults)") |
                     (cleaned_data_df ["Indicator Name"] == "Mortality rate, adult, female (per 1,000 female adults)") |
                     (cleaned_data_df ["Indicator Name"] == "Population, total") |
                     (cleaned_data_df ["Indicator Name"] == "Population, male") |
                     (cleaned_data_df ["Indicator Name"] == "Population, female") |
                     (cleaned_data_df ["Indicator Name"] == "Urban population") |
                     (cleaned_data_df ["Indicator Name"] == "Rural population") |
                     (cleaned_data_df ["Indicator Name"] == "Physicians (per 1,000 people)") |
                     (cleaned_data_df ["Indicator Name"] == "Prevalence of overweight (% of adults)") |
                     (cleaned_data_df ["Indicator Name"] == "Prevalence of overweight, male (% of male adults)") |
                     (cleaned_data_df ["Indicator Name"] == "Prevalence of overweight, female (% of female adults)")
                     , :]

#Display cleaned dataframe
cleaned_data_df.head(5)

,Country Name,Indicator Name,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
19505,Bangladesh,"Mortality rate, adult, female (per 1,000 femal...",1.358560e+02,1.315750e+02,1.279400e+02,1.243060e+02,1.206720e+02,1.170380e+02,1.134030e+02,1.102960e+02,1.071880e+02,NaN
19506,Bangladesh,"Mortality rate, adult, male (per 1,000 male ad...",1.652740e+02,1.637070e+02,1.622770e+02,1.608470e+02,1.594170e+02,1.579870e+02,1.565570e+02,1.541200e+02,1.516830e+02,NaN
19529,Bangladesh,"Physicians (per 1,000 people)",3.090000e-01,2.950000e-01,NaN,NaN,2.950000e-01,3.560000e-01,NaN,NaN,NaN,NaN
19583,Bangladesh,"Population, female",7.116678e+07,7.214503e+07,7.308747e+07,7.401692e+07,7.495057e+07,7.589301e+07,7.684136e+07,7.779618e+07,7.875587e+07,79718677.0
19585,Bangladesh,"Population, male",7.367245e+07,7.444765e+07,7.516501e+07,7.588892e+07,7.666620e+07,7.751260e+07,7.841602e+07,7.936122e+07,8.032164e+07,81276965.0


In [ ]:
#Check for missing data columns (Why is Population, total missing?)

In [ ]:
#Clean up NaN cells

In [ ]:
#Calculate and add total mortality rate to dataframe

In [ ]:
#Formatting of cell values

## Analysis
--------

#### Mortality Rate and Sex
--------

In [ ]:
#Create bar chart with mortality rate as y-axis and each of the sexes as a bar across all nations

#### Mortality Rate and Urbanization
--------

In [ ]:
#Create bar chart with mortality rate as y-axis and urban and rural populations as bars across all nations

#### Mortality Rate and Access to Physicians
--------

In [ ]:
#Create scatterplot with mortality rate as y-axis and pysicians (per 1000 people) as the x-axis across all nations

#### Mortality Rate and Obesity
--------

In [ ]:
#Create scatterplot with mortality rate as y-axis and prevalence of overweight (% of adults) as the x-axis across all nations